In [1]:
from netgen.csg import unit_cube
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh (unit_square.GenerateMesh(maxh=0.5))

class MultiGrid(BaseMatrix):
    def __init__ (self, bfa, smoothingsteps=1, cycle=1, end_level=0):
        super(MultiGrid, self).__init__()
        self.bfa = bfa
        self.inv = bfa.mat.Inverse(bfa.space.FreeDofs())
        self.mats = [bfa.mat]
        self.smoothers = [ () ]
        self.smoothingsteps = smoothingsteps
        self.cycle = cycle
        self.end_level = end_level
        
    def Update(self):
        self.mats.append (self.bfa.mat)
        blocks = []
        freedofs = self.bfa.space.FreeDofs()
        for i in range(len(freedofs)):
            if freedofs[i]:
                blocks.append ( (i,) )
        self.smoothers.append (self.bfa.mat.CreateBlockSmoother(blocks))
        
    def Height(self):
        return self.bfa.mat.height
    def Width(self):
        return self.bfa.mat.width
    
    def Mult(self, b, x):
        self.MGM(len(self.mats)-1, b, x)
        
    def MGM(self, level, b, x):
        if level > self.end_level:
            prol = self.bfa.space.Prolongation()
            nc = self.mats[level-1].height
            d = b.CreateVector()
            w = b.CreateVector()
            
            x[:] = 0
            # pre-smoothing Jacobi/Gauss-Seidel
            for i in range(self.smoothingsteps):
                self.smoothers[level].Smooth(x, b)

            # coarse grid correction
            for cgc in range(self.cycle):
                d.data = b - self.mats[level] * x
                prol.Restrict(level, d)
                self.MGM(level-1, d[0:nc] , w[0:nc])
                prol.Prolongate(level, w)
                x.data += w

            # post-smoothing Jacobi/Gauss-Seidel            
            for i in range(self.smoothingsteps):
                self.smoothers[level].SmoothBack(x, b)
        else:
            x.data = self.inv * b
            

fes = HCurl(mesh, order=1, dirichlet="left", low_order_space=False)

u,v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(u*v+curl(u)*curl(v))
a.Assemble()

pre = MultiGrid(a, smoothingsteps=1)

with TaskManager():
    for l in range(9):
        print ("level", l)
        mesh.Refine()
        fes.Update()
        print ("ndof = ", fes.ndof)
        a.Assemble()
        pre.Update()

        lam = EigenValues_Preconditioner(mat=a.mat, pre=pre)
        print ("eigenvalues: ", lam)

    


importing NGSolve-6.2.2301
level 0
ndof =  101
eigenvalues:   0.00867873
 0.0128198
 0.015554
 0.0235057
 0.0260034
 0.523099
 0.553948
 0.594216
 0.64901
 0.662635
 0.754748
 0.787064
 0.817859
 0.87304
 0.903854
 0.994372
 0.999546
 1.05519
 1.21556
 1.47669
 1.49434
   1.608

level 1
ndof =  377
eigenvalues:   0.00292292
 0.00560678
 0.0135344
 0.0245133
 0.0340472
 0.0395929
 0.524142
 0.556979
 0.565399
 0.608661
 0.634873
 0.679769
 0.733423
 0.780894
 0.82466
 0.883022
 0.913335
 0.958032
 0.989471
 0.999606
 1.07847
  1.3032
 1.64841
 1.67881
 1.85205

level 2
ndof =  1433
eigenvalues:   0.00113886
 0.00610915
 0.0121043
 0.0270808
 0.035247
 0.0413409
 0.507406
 0.523852
 0.54724
 0.585736
 0.621032
 0.674549
 0.72605
 0.77783
 0.820016
 0.873274
 0.913243
 0.956246
 0.984756
 0.999768
 1.08225
  1.3384
 1.78035
 1.81676
  1.9558

level 3
ndof =  5561
eigenvalues:   0.00106302
 0.00520611
 0.0137193
 0.0220309
 0.0407055
 0.0478107
 0.502743
 0.51208
 0.533244
 0.569211
 0.613